# 圖像相似度比較

## 載入套件

In [1]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np

## 載入VGG 16 模型

In [2]:
# 載入VGG 16 模型, 不含最上面的三層(辨識層)
model = VGG16(weights='imagenet', include_top=False)
model.summary()

58889256/58889256 [==============================] - 0s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 block1_conv1 (Conv2D)       (None, None, None, 64)    1792      
                                                                 
 block1_conv2 (Conv2D)       (None, None, None, 64)    36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, None, None, 64)    0         
                                                                 
 block2_conv1 (Conv2D)       (None, None, None, 128)   73856     
                                                                 
 block2_conv2 (Conv2D)       (None, None, None, 128)   147584    
                                                              

In [3]:
# 任選一張圖片，例如大象側面照，取得圖檔的特徵向量
img_path = './images_test/elephant.jpg'

# 載入圖檔，並縮放寬高為 (224, 224)
img = image.load_img(img_path, target_size=(224, 224))

# 加一維，變成 (1, 224, 224)
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

# 取得圖檔的特徵向量
features = model.predict(x)
print(features[0])

1/1 [==============================] - 1s 847ms/step
[[[ 0.         0.         0.        ...  0.         0.         0.       ]
  [ 0.         0.        41.877026  ...  0.         0.         0.       ]
  [ 1.0921557  0.        22.865004  ...  0.         0.         0.       ]
  ...
  [ 0.         0.         0.        ...  0.         0.         0.       ]
  [ 0.         0.         0.        ...  0.         0.         0.       ]
  [ 0.         0.         0.        ...  0.         0.         0.       ]]

 [[ 0.         0.        36.38516   ...  0.         0.         3.260647 ]
  [ 0.         0.        80.49932   ...  8.425441   0.         0.       ]
  [ 0.         0.        48.482685  ...  0.         0.         0.       ]
  ...
  [ 0.         0.         0.        ...  4.342977   0.         0.       ]
  [ 0.         0.         0.        ...  0.         0.         0.       ]
  [ 0.         0.         0.        ...  0.         0.         0.       ]]

 [[ 0.         0.        10.000878  ...  0.

In [4]:
print(features.shape)

(1, 7, 7, 512)


# 使用 cosine_similarity 比較特徵向量

### 步驟 1. 取得 images_test 目錄下所有 .jpg 檔案名稱

In [5]:
from os import listdir
from os.path import isfile, join

# 取得 images_test 目錄下所有 .jpg 檔案名稱
img_path = './images_test/'
image_files = np.array([f for f in listdir(img_path)
        if isfile(join(img_path, f)) and f[-3:] == 'jpg'])
image_files

array(['daisy1.jpg', 'tiger2.jpg', 'bird.jpg', 'tiger1.jpg', 'daisy2.jpg',
       'lion2.jpg', 'elephant2.jpg', 'deer.jpg', 'lion1.jpg',
       'elephant.jpg', 'panda1.jpg', 'bird2.jpg', 'panda2.jpg',
       'panda3.jpg', 'tiger3.jpg', 'rose2.jpg'], dtype='<U13')

### 步驟 2. 取得 images_test 目錄下所有 .jpg 檔案的像素

In [6]:
import numpy as np

# 合併所有圖檔的像素
X = np.array([])
for f in image_files:
    image_file = join(img_path, f)
    # 載入圖檔，並縮放寬高為 (224, 224)
    img = image.load_img(image_file, target_size=(224, 224))
    img2 = image.img_to_array(img)
    img2 = np.expand_dims(img2, axis=0)
    if len(X.shape) == 1:
        X = img2
    else:
        X = np.concatenate((X, img2), axis=0)

X = preprocess_input(X)

### 步驟 3. 取得所有圖檔的特徵向量

In [7]:
# 取得所有圖檔的特徵向量
features = model.predict(X)

features.shape, X.shape

1/1 [==============================] - 10s 10s/step


((16, 7, 7, 512), (16, 224, 224, 3))

### 步驟 4. 使用 cosine_similarity 函數比較特徵向量

In [8]:
# 使用 cosine_similarity 比較特徵向量
from sklearn.metrics.pairwise import cosine_similarity


# 比較 Tiger2.jpg 與其他圖檔特徵向量
no=-2
print(image_files[no])

# 轉為二維向量，類似扁平層(Flatten)
features2 = features.reshape((features.shape[0], -1))

# 排除 Tiger2.jpg 的其他圖檔特徵向量
other_features = np.concatenate((features2[:no], features2[no+1:]))

# 使用 cosine_similarity 計算 Cosine 函數
similar_list = cosine_similarity(features2[no:no+1], other_features,
                                 dense_output=False)

# 顯示相似度，由大排到小
print(np.sort(similar_list[0])[::-1])

# 依相似度，由大排到小，顯示檔名
image_files2 = np.delete(image_files, no)
image_files2[np.argsort(similar_list[0])[::-1]]

tiger3.jpg
[0.31442717 0.2666164  0.15673114 0.11975335 0.09697845 0.09630291
 0.09111855 0.08640812 0.07170606 0.06052469 0.05851331 0.0534065
 0.04204471 0.03196231 0.02107605]


array(['tiger1.jpg', 'tiger2.jpg', 'lion2.jpg', 'panda2.jpg',
       'elephant.jpg', 'lion1.jpg', 'panda1.jpg', 'panda3.jpg',
       'elephant2.jpg', 'deer.jpg', 'bird.jpg', 'rose2.jpg', 'daisy2.jpg',
       'bird2.jpg', 'daisy1.jpg'], dtype='<U13')

### 其他圖檔比較

In [9]:
# 比較對象：bird.jpg
no=0
print(image_files[no])


# 使用 cosine_similarity 計算 Cosine 函數
other_features = np.concatenate((features2[:no], features2[no+1:]))
similar_list = cosine_similarity(features2[no:no+1], other_features,
                                 dense_output=False)

# 顯示相似度，由大排到小
print(np.sort(similar_list[0])[::-1])

# 依相似度，由大排到小，顯示檔名
image_files2 = np.delete(image_files, no)
image_files2[np.argsort(similar_list[0])[::-1]]

daisy1.jpg
[0.19530812 0.05174034 0.0346877  0.03127885 0.03121524 0.02680768
 0.0259204  0.02443978 0.02309933 0.02197441 0.02107604 0.02004465
 0.01990521 0.0191945  0.01629711]


array(['daisy2.jpg', 'rose2.jpg', 'panda1.jpg', 'elephant2.jpg',
       'tiger2.jpg', 'panda3.jpg', 'bird2.jpg', 'lion1.jpg', 'tiger1.jpg',
       'elephant.jpg', 'tiger3.jpg', 'lion2.jpg', 'deer.jpg',
       'panda2.jpg', 'bird.jpg'], dtype='<U13')